# Roster Health Analysis

**Purpose**: Analyze roster composition, position depth, and age distribution for dynasty team evaluation

**Author**: FF Analytics Team

**Date**: 2024-11-08

## Objectives

- Evaluate position depth across all roster positions
- Identify age curve risks (aging RBs, young WRs with upside)
- Compare roster composition to league benchmarks
- Highlight potential holes and trade targets

In [ ]:
# Setup: Imports and configuration
import os
from pathlib import Path

import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set plotting style
sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)
plt.rcParams["font.size"] = 10

In [ ]:
# DuckDB connection setup
external_root = os.environ.get("EXTERNAL_ROOT", str(Path.cwd().parent.parent / "data" / "raw"))

db_path = Path.cwd().parent.parent / "dbt" / "ff_data_transform" / "target" / "dev.duckdb"

if db_path.exists():
    conn = duckdb.connect(str(db_path), read_only=True)
    print(f"Connected to: {db_path}")
else:
    conn = duckdb.connect()  # In-memory
    print("Using in-memory database")

# Test connection
conn.execute("SELECT 1 AS test").fetchdf()

---
**📊 Data Freshness Check**

Last updated:
- Commissioner Rosters: 2024-11-06 (2 days old)
- KTC Valuations: 2024-11-07 (1 day old)
- NFLverse Stats: 2024-11-08 (0 days old)

✅ All data sources within acceptable freshness thresholds
---

## Data Loading

Load current roster data with player metadata (age, position, KTC value)

In [ ]:
# Query current roster with player metadata
query = """
SELECT
    r.franchise_name,
    r.player_name,
    p.position,
    p.age,
    p.nfl_team,
    k.ktc_value,
    r.contract_years_remaining,
    r.salary_cap_pct
FROM main.mart_contract_snapshot_history r
JOIN main.dim_player p ON r.player_id = p.player_id
LEFT JOIN main.vw_trade_value_default k ON r.player_id = k.player_id
WHERE r.snapshot_date = (SELECT MAX(snapshot_date) FROM main.mart_contract_snapshot_history)
  AND r.franchise_name = 'My Team'  -- Replace with actual team name
  AND r.roster_status = 'active'
ORDER BY k.ktc_value DESC NULLS LAST
"""

roster_df = conn.execute(query).fetchdf()
print(f"Loaded {len(roster_df)} roster players")
roster_df.head(10)

## Position Depth Analysis

Break down roster by position to identify depth and holes

In [ ]:
# Position depth summary
position_summary = roster_df.groupby('position').agg({
    'player_name': 'count',
    'ktc_value': ['sum', 'mean'],
    'age': 'mean'
}).round(2)

position_summary.columns = ['Count', 'Total_KTC_Value', 'Avg_KTC_Value', 'Avg_Age']
position_summary = position_summary.sort_values('Total_KTC_Value', ascending=False)

print("Position Depth Summary:")
position_summary

In [ ]:
# Visualize KTC value distribution by position
fig, ax = plt.subplots(figsize=(10, 6))

sns.boxplot(data=roster_df, x='position', y='ktc_value', 
            order=['QB', 'RB', 'WR', 'TE'])

plt.title("Dynasty Value Distribution by Position")
plt.xlabel("Position")
plt.ylabel("KTC Value")
plt.tight_layout()
plt.show()

## Age Curve Analysis

Identify aging players at risk positions (RB, TE) and young players with upside (WR)

In [ ]:
# Age distribution by position
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
positions = ['QB', 'RB', 'WR', 'TE']

for idx, pos in enumerate(positions):
    ax = axes[idx // 2, idx % 2]
    pos_data = roster_df[roster_df['position'] == pos]
    
    sns.scatterplot(data=pos_data, x='age', y='ktc_value', ax=ax, s=100)
    
    # Add labels for top players
    for _, row in pos_data.nlargest(3, 'ktc_value').iterrows():
        ax.annotate(row['player_name'], (row['age'], row['ktc_value']), 
                   fontsize=8, alpha=0.7)
    
    ax.set_title(f"{pos} Age vs Value")
    ax.set_xlabel("Age")
    ax.set_ylabel("KTC Value")

plt.tight_layout()
plt.show()

In [ ]:
# Flag aging RBs (28+) with high value
aging_rbs = roster_df[
    (roster_df['position'] == 'RB') & 
    (roster_df['age'] >= 28) &
    (roster_df['ktc_value'] > 1000)
].sort_values('age', ascending=False)

if len(aging_rbs) > 0:
    print("⚠️ Aging RBs to Monitor (Age 28+):")
    print(aging_rbs[['player_name', 'age', 'ktc_value', 'nfl_team']])
else:
    print("✅ No aging RBs with significant value")

In [ ]:
# Identify young WRs (under 25) with upside
young_wrs = roster_df[
    (roster_df['position'] == 'WR') & 
    (roster_df['age'] < 25)
].sort_values('ktc_value', ascending=False)

if len(young_wrs) > 0:
    print("📈 Young WRs with Upside (Under 25):")
    print(young_wrs[['player_name', 'age', 'ktc_value', 'nfl_team']])
else:
    print("⚠️ No young WRs - consider targeting in draft/trades")

## Position Depth Charts

Show top players at each position ordered by value

In [ ]:
# Top players by position
for position in ['QB', 'RB', 'WR', 'TE']:
    print(f"\n{'='*60}")
    print(f"{position} Depth Chart")
    print(f"{'='*60}")
    
    pos_players = roster_df[
        roster_df['position'] == position
    ].sort_values('ktc_value', ascending=False)
    
    print(pos_players[['player_name', 'age', 'nfl_team', 'ktc_value']].to_string(index=False))
    print(f"\nTotal {position}s: {len(pos_players)}")
    print(f"Total Value: {pos_players['ktc_value'].sum():,.0f}")
    print(f"Average Age: {pos_players['age'].mean():.1f}")

## Conclusions

### Key Findings

1. **Position Strength**: [Identify strongest position groups by total KTC value]
2. **Depth Concerns**: [Identify positions lacking depth or quality starters]
3. **Age Curve Risks**: [Call out aging RBs or TEs that may lose value quickly]
4. **Youth Upside**: [Highlight young WRs or QBs with growth potential]

### Recommendations

- **Trade Targets**: Consider trading for [position with holes]
- **Sell Candidates**: Consider selling [aging players at peak value]
- **Draft Strategy**: Focus on [positions needing youth/depth]
- **Competitive Window**: [Win-now vs rebuild assessment based on age/value distribution]